In [13]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import os
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [14]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [15]:
def loadTrainingData():
    dataset = "Datasets"
    train = tf.keras.preprocessing.image_dataset_from_directory(
    f'{dataset}/train',
    seed=32,
    label_mode='categorical',
    image_size= IMAGE_SIZE,
    batch_size=BATCH_SIZE)
    val = tf.keras.preprocessing.image_dataset_from_directory(
    f'{dataset}/valid',
    seed=32,
    label_mode='categorical',
    image_size= IMAGE_SIZE,
    batch_size=BATCH_SIZE)
    print(train.class_names)
    return train, val


In [16]:
def createModel(train, val):
    model = Sequential()
    resnet = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),pooling='avg',classes=2)
    for layer in resnet.layers:
        layer.trainable=False
    model.add(resnet)
    model.add(layers.Flatten())
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dense(2,activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train,validation_data=val,epochs=EPOCHS)
    model.summary()
    return model



In [17]:

def classifyBees(model, train):
    predict = [] 
    label = []
    test_dir="Datasets/test/"
    for class_name in os.listdir(test_dir):
        class_path = os.path.join(test_dir, class_name)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                img=cv2.imread(img_path)
                img = resizeImage(img)
                print(img.shape)
                pred= model.predict(img)
                print(pred)
                print(f'This is {train.class_names[np.argmax(pred)]}')
                predict.append(class_name)
                label.append(train.class_names[np.argmax(pred)])
    accuracy = accuracy_score(predict, label)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    return predict, label

def resizeImage(img):
    rimg= cv2.resize(img,IMAGE_SIZE)
    img= np.expand_dims(rimg,axis=0)
    return img



In [18]:


train , val = loadTrainingData()
model = createModel(train, val)
classifyBees(model, train)




Found 219 files belonging to 2 classes.
Found 50 files belonging to 2 classes.
['bee', 'not bee']
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5945 - loss: 2.0039 - val_accuracy: 0.6800 - val_loss: 0.6411
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8121 - loss: 0.6132 - val_accuracy: 0.8400 - val_loss: 0.4123
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8750 - loss: 0.3415 - val_accuracy: 0.7200 - val_loss: 0.7406
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8758 - loss: 0.2616 - val_accuracy: 0.8400 - val_loss: 0.3819
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9702 - loss: 0.1048 - val_accuracy: 0.8200 - val_loss: 0.4877
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9921 - loss: 0.0680 - val_accuracy: 0.8200 - val_loss: 0.5706
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9857 - loss: 0.0489 - val_accuracy: 0.8400 - val_loss: 0.4416
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,738,056 (102.00 MB)

 Trainable params: 1,050,114 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 2,100,230 (8.01 MB)

(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 744ms/step
[[0.2993776 0.7006224]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0.8279046  0.17209545]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
[[0.98209494 0.01790511]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[0.9973484  0.00265151]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[[0.24126181 0.7587382 ]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
[[0.9643395  0.03566048]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[[0.00738992 0.99261004]]
This is not bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
[[9.999149e-01 8.511340e-05]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
[[0.5448793  0.45512062]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
[[0.87246066 0.12753934]]
This is bee
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


(['bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee'],
 ['not bee',
  'bee',
  'bee',
  'bee',
  'not bee',
  'bee',
  'not bee',
  'bee',
  'bee',
  'bee',
  'not bee',
  'bee',
  'not bee',
  'bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee',
  'not bee'])

In [19]:
img=cv2.imread('Datasets/test/not bee/mimicbee00018_jpeg.rf.ecfe6648e124eee71ef00fef7237b21a.jpg')
rimg= cv2.resize(img,IMAGE_SIZE)
img= np.expand_dims(rimg,axis=0)
print(img.shape)
pred= model.predict(img)
print(pred)
print(f'This is {train.class_names[np.argmax(pred)]}')

(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
[[0.25221115 0.74778885]]
This is not bee
